# Telecom Customer Churn Prediction Feature Engineering

This tutorial is based on [this](https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction/comments#6.-Model-Performances) Kaggle notebook and [this](https://github.com/gojek/feast/tree/master/examples/feast-xgboost-churn-prediction-tutorial) Feast notebook

## 1. Churn Feature Engineering

In [1]:
from hops import pandas_helper as pandas
import pandas as pd
import numpy as np
import os
import sklearn # Tested with 0.22.1
from slugify import slugify
import snowflake.connector # tested on snowflake-connector-python==2.3.1 

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
51,application_1598866185540_0015,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
import hsfs
connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store()
connector = fs.get_storage_connector("snowflake_spark_connector", "JDBC")

### 1.1 Data

In [3]:
ctx = snowflake.connector.connect(
  user='HOPSWORKS',
  password=connector.arguments,
  account='ra96958.eu-central-1',
  warehouse='HOPSWORKS_WH',
  database='ML_WORKSHOP',
  schema='PUBLIC'
)

# Query Snowflake Data
cs=ctx.cursor()
allrows=cs.execute("""select * from CUSTOMER_CHURN """).fetchall()
telcom = pd.DataFrame(allrows)
telcom.columns = ['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'churn']

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
telcom.head(5)

  customer_id  gender  senior_citizen partner dependents  tenure phone_service    multiple_lines internet_service online_security online_backup device_protection tech_support streaming_tv streaming_movies        contract paperless_billing             payment_method  monthly_charges total_charges churn
0  7590-VHVEG  Female               0     Yes         No       1            No  No phone service              DSL              No           Yes                No           No           No               No  Month-to-month               Yes           Electronic check            29.85         29.85    No
1  5575-GNVDE    Male               0      No         No      34           Yes                No              DSL             Yes            No               Yes           No           No               No        One year                No               Mailed check            56.95        1889.5    No
2  3668-QPYBK    Male               0      No         No       2           Yes             

### 1.2 Data Manipulation

In [4]:
# Replacing spaces with null values in total charges column
telcom['total_charges'] = telcom["total_charges"].replace(" ",np.nan)

# Dropping null values from total charges column which contain .15% missing data 
telcom = telcom[telcom["total_charges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

# Convert to float type
telcom["total_charges"] = telcom["total_charges"].astype(float)

# Replace 'No internet service' to No for the following columns
replace_cols = [ 'online_security', 'online_backup', 'device_protection',
                'tech_support','streaming_tv', 'streaming_movies']
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'internet_service' : 'No'})
    
# Replace binary values with strings
telcom["senior_citizen"] = telcom["senior_citizen"].replace({1:"Yes",0:"No"})

# Tenure to categorical column
def tenure_lab(telcom) :
    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom),
                                      axis = 1)

# Separating churn and non churn customers
churn     = telcom[telcom["churn"] == "Yes"]
not_churn = telcom[telcom["churn"] == "No"]

# Separating catagorical and numerical columns
Id_col     = ['customer_id']
target_col = ["churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

telcom.head()

  customer_id  gender senior_citizen partner dependents  tenure phone_service    multiple_lines internet_service online_security online_backup device_protection tech_support streaming_tv streaming_movies        contract paperless_billing             payment_method  monthly_charges  total_charges churn  tenure_group
0  7590-VHVEG  Female             No     Yes         No       1            No  No phone service              DSL              No           Yes                No           No           No               No  Month-to-month               Yes           Electronic check            29.85          29.85    No   Tenure_0-12
1  5575-GNVDE    Male             No      No         No      34           Yes                No              DSL             Yes            No               Yes           No           No               No        One year                No               Mailed check            56.95        1889.50    No  Tenure_24-48
2  3668-QPYBK    Male             No      No     

### 1.3 Data Processing

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Customer id col
Id_col     = ['customer_id']

# Target columns
target_col = ["churn"]

# Categorical columns
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]

# Numerical columns
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

# Binary columns with 2 values
bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()

# Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

# Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])
    
# Duplicating columns for multi value columns
telcom = pd.get_dummies(data = telcom,columns = multi_cols )

# Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

# Dropping original values merging scaled values for numerical columns
df_telcom_og = telcom.copy()
telcom = telcom.drop(columns = num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")

# Clean up column names
telcom.columns = [slugify(col, lowercase=True, separator='_') for col in telcom.columns]
telcom.columns = map(str.lower, telcom.columns)
telcom.head()

  customer_id  gender  senior_citizen  partner  dependents  phone_service  paperless_billing  churn  multiple_lines_no  multiple_lines_no_phone_service  multiple_lines_yes  internet_service_dsl  internet_service_fiber_optic  internet_service_no  online_security_no  online_security_no_internet_service  online_security_yes  online_backup_no  online_backup_no_internet_service  online_backup_yes  device_protection_no  device_protection_no_internet_service  device_protection_yes  tech_support_no  tech_support_no_internet_service  tech_support_yes  streaming_tv_no  streaming_tv_no_internet_service  streaming_tv_yes  streaming_movies_no  streaming_movies_no_internet_service  streaming_movies_yes  contract_month_to_month  contract_one_year  contract_two_year  payment_method_bank_transfer_automatic  payment_method_credit_card_automatic  payment_method_electronic_check  payment_method_mailed_check  tenure_group_tenure_0_12  tenure_group_tenure_12_24  tenure_group_tenure_24_48  tenure_group_tenur

### 1.4 Create FeatureGroups

In [ ]:
val telcoFg = (fs.createFeatureGroup()
                 .name("telco_customer_features")
                 .version(2)
                 .description("Telecom customer features")
                 .onlineEnabled(true)
                 .timeTravelFormat(TimeTravelFormat.HUDI)
                 .primaryKeys(Seq("customer_id"))
                 .statisticsEnabled(true)
                 .build())

telcoFg.save(telcom_churn_features)